# Mapping Chinese Dialects in the U.S.

## Data Processing
Data is presented as an excel workbook, where each sheet is a county. Let's parse this into a more usable csv format

In [1]:
import pandas as pd

In [8]:
counties = pd.read_excel(r'2009-2013-acs-lang-tables-county.xls', sheet_name='Contents')
counties.head()

,Table,County
0,1,"Anchorage Municipality, AK"
1,2,"Maricopa County, AZ"
2,3,"Navajo County, AZ"
3,4,"Pima County, AZ"
4,5,"Alameda County, CA"


In [50]:
language_by_county = []

for i, row in counties.iterrows():
    print(i, row["County"])
    county_df = pd.read_excel(r'2009-2013-acs-lang-tables-county.xls', sheet_name=row["County"], skiprows=4)
    county_df.columns = ["Language", "Number of Speakers", "Margin of Error 1", "Speak English Less Than Very Well", "Margin of Error 2"]

    # Get all rows for Chinese Language 
    in_correct_section = False 
    chinese_languages = []

    for i, r in county_df.iterrows():
        # Find the rows which contain Chinese language, and Chinese dialect data
        if type(r["Language"]) == str and r["Language"].startswith(".Chinese"): in_correct_section = True
        if type(r["Language"]) == str and in_correct_section and not r["Language"].startswith("..") and not r["Language"].startswith(".Chinese"): in_correct_section = False 

        if in_correct_section: chinese_languages.append({
            "language": r["Language"],
            "speakers": r["Number of Speakers"],
            "speak_english_less_than_very_well": r["Speak English Less Than Very Well"]
        })

    language_by_county.append((row["County"], chinese_languages))


0 Anchorage Municipality, AK
1 Maricopa County, AZ
2 Navajo County, AZ
3 Pima County, AZ
4 Alameda County, CA
5 Contra Costa County, CA
6 Fresno County, CA
7 Kern County, CA
8 Los Angeles County, CA
9 Monterey County, CA
10 Orange County, CA
11 Placer County, CA
12 Riverside County, CA
13 Sacramento County, CA
14 San Bernardino County, CA
15 San Diego County, CA
16 San Francisco County, CA
17 San Joaquin County, CA
18 San Mateo County, CA
19 Santa Barbara County, CA
20 Santa Clara County, CA
21 Solano County, CA
22 Sonoma County, CA
23 Stanislaus County, CA
24 Ventura County, CA
25 Yolo County, CA
26 Arapahoe County, CO
27 Denver County, CO
28 El Paso County, CO
29 Fairfield County, CT
30 Hartford County, CT
31 New Haven County, CT
32 New Castle County, DE
33 District of Columbia, DC
34 Broward County, FL
35 Collier County, FL
36 Duval County, FL
37 Hillsborough County, FL
38 Lee County, FL
39 Miami-Dade County, FL
40 Orange County, FL
41 Palm Beach County, FL
42 Pinellas County, FL
43

In [51]:
language_by_county

[('Anchorage Municipality, AK',
  [{'language': '.Chinese (incl. Cantonese, Mandarin, other Chinese languages4)',
    'speakers': 882,
    'speak_english_less_than_very_well': 519},
   {'language': '..Chinese',
    'speakers': 740,
    'speak_english_less_than_very_well': 445},
   {'language': '..Cantonese',
    'speakers': 40,
    'speak_english_less_than_very_well': '(B)'},
   {'language': '..Mandarin',
    'speakers': 95,
    'speak_english_less_than_very_well': 65},
   {'language': '..Formosan',
    'speakers': '(D)',
    'speak_english_less_than_very_well': '(D)'}]),
 ('Maricopa County, AZ',
  [{'language': '.Chinese (incl. Cantonese, Mandarin, other Chinese languages4)',
    'speakers': 20337,
    'speak_english_less_than_very_well': 10163},
   {'language': '..Chinese',
    'speakers': 13750,
    'speak_english_less_than_very_well': 7445},
   {'language': '..Hakka',
    'speakers': '(D)',
    'speak_english_less_than_very_well': '(B)'},
   {'language': '..Cantonese',
    'speaker

In [63]:
# Get all dialects in census data
dialects = set()
for county, languages in language_by_county: 
    for language in languages: 
        dialects.add(language["language"])
        
dialects = sorted(list(dialects), reverse = True)
dialects

['.Chinese (incl. Cantonese, Mandarin, other Chinese languages4)',
 '..Wu',
 '..Mandarin',
 '..Kan, Hsiang',
 '..Hakka',
 '..Fuchow',
 '..Formosan',
 '..Chinese',
 '..Cantonese']

In [72]:
# Let's create a dataframe with the data now 
rows = []

for county, languages in language_by_county: 
    row = {"County": county}
    
    for language in languages: 
        row[language["language"]] = language["speakers"] if language["speakers"] != "(D)" else None

    rows.append(row)
    
language_df = pd.DataFrame(rows, columns = ["County"] + dialects)

In [73]:
language_df.head()

,County,".Chinese (incl. Cantonese, Mandarin, other Chinese languages4)",..Wu,..Mandarin,"..Kan, Hsiang",..Hakka,..Fuchow,..Formosan,..Chinese,..Cantonese
0,"Anchorage Municipality, AK",882,NaN,95.0,NaN,NaN,NaN,NaN,740.0,40.0
1,"Maricopa County, AZ",20337,NaN,3890.0,NaN,NaN,NaN,450.0,13750.0,2235.0
2,"Navajo County, AZ",35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Pima County, AZ",6334,NaN,955.0,NaN,NaN,NaN,125.0,4625.0,630.0
4,"Alameda County, CA",119759,135.0,17450.0,NaN,45.0,NaN,1915.0,69165.0,31050.0


In [74]:
# Let's make these columns more readable 
language_df.columns = [d.replace(".", "") for d in language_df.columns]
language_df.rename(columns={'Chinese (incl Cantonese, Mandarin, other Chinese languages4)':'Chinese (Total)'}, inplace=True)
language_df.head()

,County,Chinese (Total),Wu,Mandarin,"Kan, Hsiang",Hakka,Fuchow,Formosan,Chinese,Cantonese
0,"Anchorage Municipality, AK",882,NaN,95.0,NaN,NaN,NaN,NaN,740.0,40.0
1,"Maricopa County, AZ",20337,NaN,3890.0,NaN,NaN,NaN,450.0,13750.0,2235.0
2,"Navajo County, AZ",35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Pima County, AZ",6334,NaN,955.0,NaN,NaN,NaN,125.0,4625.0,630.0
4,"Alameda County, CA",119759,135.0,17450.0,NaN,45.0,NaN,1915.0,69165.0,31050.0


In [75]:
# Save the data 
language_df.to_csv("languages.csv")